# IBS of 3 bars - Strategy - Backtesting 

### Import Library

In [21]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [22]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [23]:
data = dataset.copy()

In [24]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [25]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627


In [26]:
def get_signal(r):
    signal = ''
    if r['ibs_s2'] > r['ibs_s1'] > r['ibs']:
        # ibs giam dan
        signal = 'short'
    elif r['ibs_s2'] < r['ibs_s1'] < r['ibs']:
        # ibs tang dan
        signal = 'long'
    return signal

In [27]:
def prepare_data(data):
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data['ibs'] = data.apply(lambda x: (-1.0 if (x["High"] == x["Low"]) else (x["Close"] - x["Low"]) / (x["High"] - x["Low"])), axis=1)
    data['ibs_s1'] = data['ibs'].shift(1)
    data['ibs_s2'] = data['ibs'].shift(2)
    data['signal'] = data.apply(lambda r: get_signal(r), axis=1)
    return data

In [28]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [29]:
prepared_data

,Open,High,Low,Close,Volume,max_5,min_5,ibs,ibs_s1,ibs_s2,signal
Date,,,,,,,,,,,
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939,900.2,896.5,0.809524,0.307692,0.500000,
2020-11-02 09:25:00,898.3,898.6,897.8,898.5,1607,900.2,896.5,0.875000,0.809524,0.307692,long
2020-11-02 09:30:00,898.5,899.4,898.3,899.0,1623,900.0,896.5,0.636364,0.875000,0.809524,
2020-11-02 09:35:00,899.1,899.8,898.7,898.9,2180,899.8,896.5,0.181818,0.636364,0.875000,short
2020-11-02 09:40:00,899.0,899.4,898.7,898.8,1190,899.8,896.5,0.142857,0.181818,0.636364,short
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524,1298.7,1295.9,0.187500,0.823529,0.520000,
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161,1298.7,1295.9,-1.000000,0.187500,0.823529,short
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627,1298.7,1296.2,-1.000000,-1.000000,0.187500,


In [35]:
prepared_data[prepared_data.signal != ''].tail(20)

,Open,High,Low,Close,Volume,max_5,min_5,ibs,ibs_s1,ibs_s2,signal
Date,,,,,,,,,,,
2024-11-22 13:25:00,1300.9,1301.8,1299.6,1299.6,6766,1301.8,1295.9,0.000000,0.809524,0.923077,short
2024-11-22 13:35:00,1298.3,1299.0,1298.2,1299.0,3647,1301.8,1296.1,1.000000,0.083333,0.000000,long
2024-11-22 13:45:00,1300.0,1300.0,1294.2,1295.0,10516,1301.8,1294.2,0.137931,0.666667,1.000000,short
2024-11-22 14:00:00,1294.7,1294.7,1291.3,1292.3,11661,1300.2,1291.3,0.294118,0.411765,1.000000,short
2024-11-22 14:15:00,1295.7,1296.1,1294.1,1294.7,5895,1296.6,1291.3,0.300000,0.590909,0.612903,short
2024-11-22 14:30:00,1294.5,1294.5,1294.5,1294.5,216,1296.6,1292.2,-1.000000,0.100000,0.777778,short
2024-11-25 09:20:00,1297.9,1298.2,1296.2,1296.6,4172,1300.0,1296.2,0.200000,0.250000,0.266667,short
2024-11-25 09:30:00,1297.3,1298.0,1297.1,1297.8,2069,1298.9,1296.2,0.777778,0.500000,0.200000,long
2024-11-25 09:40:00,1297.5,1297.5,1295.7,1295.9,3577,1298.3,1295.7,0.111111,0.181818,0.777778,short


In [31]:
class MainStrategy(Strategy):
    max_sl = 3.1
    trailing_sl = 4.5
    tp_step = 12
    def init(self):
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
                return
                
        if current_time.hour == 14 and current_time.minute >= 30:
            return

        close_price = self.data.Close[-1]
        if self.position.is_long:
            max_5 = self.data.max_5[-1]
            if close_price < max_5 - self.trailing_sl:
                self.position.close()
        elif self.position.is_short:
            min_5 = self.data.min_5[-1]
            if close_price > min_5 + self.trailing_sl:
                self.position.close()
 
        if self.position:
            return  
        signal = self.data.signal[-1]
        if signal == 'long':
            buy_price = close_price
            sl = buy_price - self.max_sl
            tp = buy_price + self.tp_step
            self.buy(size=1, sl=sl, tp=tp)
        elif signal == 'short':
            sell_price = close_price
            sl = sell_price + self.max_sl
            tp = sell_price - self.tp_step
            self.sell(size=1, sl=sl, tp=tp)

In [32]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [33]:
stats

Start                     2020-11-02 09:20:00
End                       2024-11-26 09:05:00
Duration                   1484 days 23:45:00
Exposure Time [%]                   77.484262
Equity Final [$]                   9283.17183
Equity Peak [$]                   10119.08746
Return [%]                          -7.168282
Buy & Hold Return [%]               44.533511
Return (Ann.) [%]                   -1.842405
Volatility (Ann.) [%]                1.634295
Sharpe Ratio                        -1.127339
Sortino Ratio                        -1.55049
Calmar Ratio                         -0.22279
Max. Drawdown [%]                   -8.269675
Avg. Drawdown [%]                   -0.335789
Max. Drawdown Duration     1300 days 22:35:00
Avg. Drawdown Duration       42 days 04:43:00
# Trades                                 4666
Win Rate [%]                        33.261895
Best Trade [%]                       1.372462
Worst Trade [%]                     -0.920813
Avg. Trade [%]                    

In [34]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,2,41,898.76955,898.0,-0.76955,-0.000856,2020-11-02 09:30:00,2020-11-02 14:10:00,None,0 days 04:40:00
1,1,43,45,899.26970,906.7,7.43030,0.008263,2020-11-02 14:20:00,2020-11-02 14:30:00,None,0 days 00:10:00
2,1,49,81,909.07264,905.7,-3.37264,-0.003710,2020-11-03 09:10:00,2020-11-03 13:15:00,None,0 days 04:05:00
3,-1,82,93,906.42799,909.8,-3.37201,-0.003720,2020-11-03 13:20:00,2020-11-03 14:15:00,None,0 days 00:55:00
4,-1,96,97,908.72730,908.3,0.42730,0.000470,2020-11-03 14:30:00,2020-11-03 14:45:00,None,0 days 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...
4661,-1,51712,51717,1296.91081,1300.5,-3.58919,-0.002767,2024-11-22 11:30:00,2024-11-22 13:20:00,None,0 days 01:50:00
4662,-1,51719,51731,1299.31009,1294.5,4.81009,0.003702,2024-11-22 13:30:00,2024-11-22 14:30:00,None,0 days 01:00:00
4663,-1,51738,51761,1296.21102,1299.7,-3.48898,-0.002692,2024-11-25 09:25:00,2024-11-25 11:20:00,None,0 days 01:55:00
4664,-1,51762,51774,1296.81084,1300.1,-3.28916,-0.002536,2024-11-25 11:25:00,2024-11-25 13:50:00,None,0 days 02:25:00
